In [200]:
#get data on ways fighter won/lost through dec, won/lost through sub, won/lost through ko/tko  
#tabulate outcomes in a table such as: Outcome, % chance, winner 
#FRONT END: give the user the ability to input two fighters (drop down menu)
# ---- FINISHED --- 
#extra: prevent users from picking two fighters that are from different weight classes
#--- LONG RUN --- 
# Improve model where it takes # of strikes (successful, attempted) , ground game/submissions duration, takedowns (successful, attempted)
# into consideration 

In [201]:
#get accuracy of model by spitting data into train/test
import pandas as pd 

In [202]:
df = pd.read_csv("UFC Matches April 2023.csv", )

In [203]:
df.head()

,Full Name,Weight,Matchup,Outcome,Method,Round,Time
0,Tom Aaron,WEIGHT: 155 lbs.,NaN,NaN,NaN,NaN,NaN
1,Tom Aaron,WEIGHT: 155 lbs.,Tom Aaron\n \n Matt Ricehouse,LOSS,U-DEC,3.0,5:00
2,Tom Aaron,WEIGHT: 155 lbs.,Tom Aaron\n \n Eric Steenberg,WIN,SUB,1.0,0:56
3,Danny Abbadi,WEIGHT: 155 lbs.,NaN,NaN,NaN,NaN,NaN
4,Danny Abbadi,WEIGHT: 155 lbs.,Danny Abbadi\n \n Jorge Gurgel,LOSS,S-DEC,3.0,5:00


## Data Cleaning

We have to clean our dataset then group each fighter and get their distributions to determine their likelihood of winning/losing via decision, tko/ko or submission.

In [204]:
# delete first row of every fighter
nulls = df[["Matchup", "Outcome", 'Method', 'Round', 'Time']].isnull().all(axis=1) 
df = df.loc[~nulls, :]

#find matches that are upcoming and delete them
df = df[df["Outcome"] != 'NEXT']

# delete "weight text" in weight column
df['Weight'] = df['Weight'].str.replace('WEIGHT: ', '')

# keep only opponents name in matchup column
def remove_name(orig_matchup):
    last_index = orig_matchup.rfind('\n')
    new_string = orig_matchup[last_index +1:]
    return new_string.strip()

df['Matchup'] = df['Matchup'].apply(lambda x: remove_name(x))

In [205]:
unique_methods = df['Method'].unique()

In [206]:
unique_methods

array(['U-DEC', 'SUB', 'S-DEC', 'KO/TKO', 'Overturned', 'Other', 'M-DEC',
       'CNC', 'DQ', 'SUB ', 'KO/TKO ', 'S-DEC ', 'Decision', 'U-DEC ',
       'M-DEC ', 'Overturned ', 'CNC '], dtype=object)

In [207]:
unique_outcomes = df['Outcome'].unique()

In [208]:
unique_outcomes

array(['LOSS', 'WIN', 'NC', 'DRAW'], dtype=object)

In [209]:
removed_outcomes = ['NC', 'DRAW']
removed_methods = ['Overturned', 'Other', 'CNC', 'DQ', 'Overturned ', 'CNC ']

In [210]:
filtered_df1 = df.loc[~df['Method'].isin(removed_methods)]
filtered_df2 = filtered_df1.loc[~df['Outcome'].isin(removed_outcomes)]

In [211]:
filtered_df2

,Full Name,Weight,Matchup,Outcome,Method,Round,Time
1,Tom Aaron,155 lbs.,Matt Ricehouse,LOSS,U-DEC,3.0,5:00
2,Tom Aaron,155 lbs.,Eric Steenberg,WIN,SUB,1.0,0:56
4,Danny Abbadi,155 lbs.,Jorge Gurgel,LOSS,S-DEC,3.0,5:00
5,Danny Abbadi,155 lbs.,Kalib Starnes,LOSS,SUB,1.0,2:56
7,Nariman Abbasov,155 lbs.,Ismael Bonfim,LOSS,U-DEC,3.0,5:00
...,...,...,...,...,...,...,...
22976,Alex Zuniga,145 lbs.,Bryan Caraway,LOSS,U-DEC,3.0,5:00
22979,Allan Zuniga,155 lbs.,John Gunther,LOSS,M-DEC,3.0,5:00
22981,Virgil Zwicker,205 lbs.,Guto Inocente,LOSS,U-DEC,3.0,5:00
22982,Virgil Zwicker,205 lbs.,Brett Albee,WIN,KO/TKO,1.0,1:46


In [212]:
df = filtered_df2

In [213]:
decisions = ['U-DEC', 'S-DEC', 'M-DEC', 'S-DEC ', 'U-DEC ', 'M-DEC ']
kos = ['KO/TKO', 'KO/TKO ']
submissions = ['SUB', 'SUB ']

In [214]:
decision_df = df[df['Method'].isin(decisions)]
decision_count = decision_df['Method'].count()

ko_df = df[df['Method'].isin(kos)]
ko_count = ko_df['Method'].count()

submission_df = df[df['Method'].isin(submissions)]
submission_count = submission_df['Method'].count()

print(decision_count/len(df))
print(ko_count/len(df))
print(submission_count/len(df))

0.4286023366508005
0.34714409346603203
0.223929035049762


## Exploratory Data Analysis (EDA)

In [215]:
#questions that could be of importance:
#what is the most common method of winning?
#for fighters that win through ko/submission what round does it typically occur in?
#is there a discrepancy in the way fighters win across different weight classes?

In [216]:
#Jan vs. Izzy:

#Outcome	% chance	Winner
#KO	44.9%	Jan
#DEC	29.1%	Israel
#DEC	13.2%	Jan
#KO	9.48%	Israel
#SUB	2.9%	Israel
#SUB	.4%	Jan

In [217]:
#name, total matches, wins, losses, wins by dec, wins by sub, wins by ko/tko, loss by dec, loss by sub, loss by ko/tko

In [218]:
data = {
    'Total Fights': df.groupby(['Full Name'])['Matchup'].count(),
    'Total Wins': df.loc[df['Outcome'] == 'WIN'].groupby(['Full Name'])['Outcome'].count(),
    'Total Losses': df.loc[df['Outcome'] == 'LOSS'].groupby(['Full Name'])['Outcome'].count(),
    'Wins By Decision': df.loc[(df['Outcome'] == 'WIN') & (df['Method'].isin(decisions))].groupby(['Full Name'])['Outcome'].count(),
    'Wins By KO': df.loc[(df['Outcome'] == 'WIN') & (df['Method'].isin(kos))].groupby(['Full Name'])['Outcome'].count(),
    'Wins By Submission': df.loc[(df['Outcome'] == 'WIN') & (df['Method'].isin(submissions))].groupby(['Full Name'])['Outcome'].count(),
    'Loss By Decision': df.loc[(df['Outcome'] == 'LOSS') & (df['Method'].isin(decisions))].groupby(['Full Name'])['Outcome'].count(),
    'Loss By KO': df.loc[(df['Outcome'] == 'LOSS') & (df['Method'].isin(kos))].groupby(['Full Name'])['Outcome'].count(),
    'Loss By Submission': df.loc[(df['Outcome'] == 'LOSS') & (df['Method'].isin(submissions))].groupby(['Full Name'])['Outcome'].count()
}

In [219]:
fighter_stats = pd.DataFrame(data)

In [220]:
#df[df['Full Name'] == 'AJ Fonseca']

In [221]:
fighter_stats.fillna(0, inplace=True)

In [222]:
fighter_stats

,Total Fights,Total Wins,Total Losses,Wins By Decision,Wins By KO,Wins By Submission,Loss By Decision,Loss By KO,Loss By Submission
AJ Dobson,3,1.0,2.0,0.0,0.0,1.0,2.0,0.0,0.0
AJ Fletcher,4,2.0,2.0,0.0,1.0,1.0,2.0,0.0,0.0
AJ Fonseca,1,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
AJ Matthews,1,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
Aalon Cruz,3,1.0,2.0,0.0,1.0,0.0,0.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...
Zhang Tiequan,6,2.0,4.0,0.0,0.0,2.0,3.0,1.0,0.0
Zhang Weili,9,7.0,2.0,3.0,2.0,2.0,1.0,1.0,0.0
Zoila Frausto,2,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
Zubaira Tukhugov,8,5.0,3.0,3.0,2.0,0.0,3.0,0.0,0.0


In [223]:
df = fighter_stats

## Monte Carlo Simulation

In [224]:
import math
import random as rnd

In [225]:
def calculate_mean(statistic, total_fights):
    return statistic/total_fights

def calculate_sd(statistic, total_fights):
    mean = calculate_mean(statistic, total_fights)
    return math.sqrt((mean*(1-mean))/total_fights)

In [289]:
def get_fighter_parameters(fighter1, fighter2):
    data = {"Decision Wins Mean" : [calculate_mean(df.loc[fighter1, "Wins By Decision"], df.loc[fighter1, "Total Fights"]),
                                   calculate_mean(df.loc[fighter2, "Wins By Decision"], df.loc[fighter2, "Total Fights"])], 
            
           "Decision Wins SD" : [calculate_sd(df.loc[fighter1, "Wins By Decision"], df.loc[fighter1, "Total Fights"]),
                                   calculate_sd(df.loc[fighter2, "Wins By Decision"], df.loc[fighter2, "Total Fights"])],
            
           "KO Wins Mean" : [calculate_mean(df.loc[fighter1, "Wins By KO"], df.loc[fighter1, "Total Fights"]),
                                   calculate_mean(df.loc[fighter2, "Wins By KO"], df.loc[fighter2, "Total Fights"])], 
            
            'KO Wins SD' : [calculate_sd(df.loc[fighter1, "Wins By KO"], df.loc[fighter1, "Total Fights"]),
                                   calculate_sd(df.loc[fighter2, "Wins By KO"], df.loc[fighter2, "Total Fights"])], 
           
            'Sub Wins Mean' : [calculate_mean(df.loc[fighter1, "Wins By Submission"], df.loc[fighter1, "Total Fights"]),
                                   calculate_mean(df.loc[fighter2, "Wins By Submission"], df.loc[fighter2, "Total Fights"])],
           
            'Sub Wins SD' : [calculate_sd(df.loc[fighter1, "Wins By Submission"], df.loc[fighter1, "Total Fights"]),
                                   calculate_sd(df.loc[fighter2, "Wins By Submission"], df.loc[fighter2, "Total Fights"])], 
            
            'Decision Loss Mean' : [calculate_mean(df.loc[fighter1, "Loss By Decision"], df.loc[fighter1, "Total Fights"]),
                                   calculate_mean(df.loc[fighter2, "Loss By Decision"], df.loc[fighter2, "Total Fights"])], 
            
            'Decision Loss SD' : [calculate_sd(df.loc[fighter1, "Loss By Decision"], df.loc[fighter1, "Total Fights"]),
                                   calculate_sd(df.loc[fighter2, "Loss By Decision"], df.loc[fighter2, "Total Fights"])], 
            
            'KO Loss Mean' : [calculate_mean(df.loc[fighter1, "Loss By KO"], df.loc[fighter1, "Total Fights"]),
                                   calculate_mean(df.loc[fighter2, "Loss By KO"], df.loc[fighter2, "Total Fights"])], 
            
            'KO Loss SD' : [calculate_sd(df.loc[fighter1, "Loss By KO"], df.loc[fighter1, "Total Fights"]),
                                   calculate_sd(df.loc[fighter2, "Loss By KO"], df.loc[fighter2, "Total Fights"])], 
           
            'Sub Loss Mean' : [calculate_mean(df.loc[fighter1, "Loss By Submission"], df.loc[fighter1, "Total Fights"]),
                                   calculate_mean(df.loc[fighter2, "Loss By Submission"], df.loc[fighter2, "Total Fights"])], 
            
            'Sub Loss SD' : [calculate_sd(df.loc[fighter1, "Loss By Submission"], df.loc[fighter1, "Total Fights"]),
                                   calculate_sd(df.loc[fighter2, "Loss By Submission"], df.loc[fighter2, "Total Fights"])]}
    
    fighter_parameter_df = pd.DataFrame(data=data, index = [fighter1, fighter2])
    return fighter_parameter_df

In [322]:
matchup_df = get_fighter_parameters("Mateusz Gamrot", "Arman Tsarukyan")

In [317]:
#make multipliers?
#0.4286023366508005 --> 43% end by decision
#0.34714409346603203 --> 35% end by ko
#0.223929035049762 --> 22% end by submission

In [226]:
def gameSim():
    results = []
    fighter1_dec_score = (rnd.gauss(matchup_df.iloc[0]['Decision Wins Mean'],matchup_df.iloc[0]['Decision Wins SD'])+ 
                          rnd.gauss(matchup_df.iloc[1]['Decision Loss Mean'],matchup_df.iloc[1]['Decision Loss SD']))/2
    fighter1_ko_score = (rnd.gauss(matchup_df.iloc[0]['KO Wins Mean'],matchup_df.iloc[0]['KO Wins SD'])+ 
                          rnd.gauss(matchup_df.iloc[1]['KO Loss Mean'],matchup_df.iloc[1]['KO Loss SD']))/2
    fighter1_sub_score = (rnd.gauss(matchup_df.iloc[0]['Sub Wins Mean'],matchup_df.iloc[0]['Sub Wins SD'])+ 
                          rnd.gauss(matchup_df.iloc[1]['Sub Loss Mean'],matchup_df.iloc[1]['Sub Loss SD']))/2
    fighter2_dec_score = (rnd.gauss(matchup_df.iloc[1]['Decision Wins Mean'],matchup_df.iloc[1]['Decision Wins SD'])+ 
                          rnd.gauss(matchup_df.iloc[0]['Decision Loss Mean'],matchup_df.iloc[0]['Decision Loss SD']))/2
    fighter2_ko_score = (rnd.gauss(matchup_df.iloc[1]['KO Wins Mean'],matchup_df.iloc[1]['KO Wins SD'])+ 
                          rnd.gauss(matchup_df.iloc[0]['KO Loss Mean'],matchup_df.iloc[0]['KO Loss SD']))/2
    fighter2_sub_score = (rnd.gauss(matchup_df.iloc[1]['Sub Wins Mean'],matchup_df.iloc[1]['Sub Wins SD'])+ 
                          rnd.gauss(matchup_df.iloc[0]['Sub Loss Mean'],matchup_df.iloc[0]['Sub Loss SD']))/2
    
    #results.append(fighter1_dec_score*0.5713976633491995)
    #results.append(fighter1_ko_score*0.652855906533968)
    #results.append(fighter1_sub_score*0.776070964950238)    
    #results.append(fighter2_dec_score*0.5713976633491995)    
    #results.append(fighter2_ko_score*0.652855906533968)    
    #results.append(fighter2_sub_score*0.776070964950238)     
    
    results.append(fighter1_dec_score)
    results.append(fighter1_ko_score)
    results.append(fighter1_sub_score)    
    results.append(fighter2_dec_score)    
    results.append(fighter2_ko_score)    
    results.append(fighter2_sub_score)
    
    #print(results)
    if max(results) == results[0]:
        return "f1_dec"
    elif max(results) == results[1]:
        return "f1_ko"
    elif max(results) == results[2]:
        return "f1_sub"
    elif max(results) == results[3]:
        return "f2_dec"
    elif max(results) == results[4]:
        return "f2_ko"
    elif max(results) == results[5]:
        return "f2_sub"
    else: return "tie"

In [227]:
def gamesSim(ns):
    matchesout = []
    fighter1_decwin = 0
    fighter1_kowin = 0
    fighter1_subwin = 0
    fighter2_decwin = 0
    fighter2_kowin = 0
    fighter2_subwin = 0
    tie = 0
    for i in range(ns):
        gm = gameSim()
        matchesout.append(gm)
        if gm == "f1_dec":
            fighter1_decwin +=1 
        elif gm == "f1_ko":
            fighter1_kowin +=1 
        elif gm == "f1_sub":
            fighter1_subwin +=1 
        elif gm == "f2_dec":
            fighter2_decwin +=1 
        elif gm == "f2_ko":
            fighter2_kowin +=1 
        elif gm == "f2_sub":
            fighter2_subwin +=1 
        else: tie +=1 
    
    print(matchup_df.index[0] +' Decision Win ', round((fighter1_decwin/ns)*100,2),'%')
    print(matchup_df.index[0] +' KO Win ', round((fighter1_kowin/ns)*100,2),'%')
    print(matchup_df.index[0] +' Submission Win ', round((fighter1_subwin/ns)*100,2),'%')
    print(matchup_df.index[1] +' Decision Win ', round((fighter2_decwin/ns)*100,2),'%')
    print(matchup_df.index[1] +' KO Win ', round((fighter2_kowin/ns)*100,2),'%')
    print(matchup_df.index[1] +' Submission Win ', round((fighter2_subwin/ns)*100,2),'%')
    print('Tie ', (tie/ns)*100, '%')
    
    #return matchesout

## Predictions on UFC Fight Night: Song vs Simón

### Song Yadong vs Ricky Simón

In [229]:
matchup_df = get_fighter_parameters("Song Yadong", "Ricky Simon")
gamesSim(10000)

Song Yadong Decision Win  13.7 %
Song Yadong KO Win  32.91 %
Song Yadong Submission Win  0.05 %
Ricky Simon Decision Win  50.26 %
Ricky Simon KO Win  0.62 %
Ricky Simon Submission Win  2.46 %
Tie  0.0 %


### Caio Borralho vs Michal Oleksiejczuk

In [231]:
matchup_df = get_fighter_parameters("Caio Borralho", "Michal Oleksiejczuk")
gamesSim(10000)

Caio Borralho Decision Win  90.54 %
Caio Borralho KO Win  0.19 %
Caio Borralho Submission Win  0.06 %
Michal Oleksiejczuk Decision Win  0.0 %
Michal Oleksiejczuk KO Win  9.21 %
Michal Oleksiejczuk Submission Win  0.0 %
Tie  0.0 %


### Rodolfo Vieira vs Cody Brundage

In [232]:
matchup_df = get_fighter_parameters("Rodolfo Vieira", "Cody Brundage")
gamesSim(10000)

Rodolfo Vieira Decision Win  2.14 %
Rodolfo Vieira KO Win  17.2 %
Rodolfo Vieira Submission Win  56.09 %
Cody Brundage Decision Win  2.25 %
Cody Brundage KO Win  2.08 %
Cody Brundage Submission Win  20.24 %
Tie  0.0 %


### Julian Erosa vs Fernando Padilla - N/A (Fernando Padilla's first fight recorded on fightmetric)

### Marcos Rogério de Lima vs Waldo Cortes-Acosta

In [238]:
matchup_df = get_fighter_parameters("Marcos Rogerio de Lima", "Cody Brundage")
gamesSim(10000)

Marcos Rogerio de Lima Decision Win  5.49 %
Marcos Rogerio de Lima KO Win  69.18 %
Marcos Rogerio de Lima Submission Win  0.01 %
Cody Brundage Decision Win  0.02 %
Cody Brundage KO Win  1.38 %
Cody Brundage Submission Win  23.92 %
Tie  0.0 %


### Josh Quinlan vs Trey Waters

In [239]:
matchup_df = get_fighter_parameters("Josh Quinlan", "Trey Waters")
gamesSim(10000)

Josh Quinlan Decision Win  0.0 %
Josh Quinlan KO Win  100.0 %
Josh Quinlan Submission Win  0.0 %
Trey Waters Decision Win  0.0 %
Trey Waters KO Win  0.0 %
Trey Waters Submission Win  0.0 %
Tie  0.0 %


### Martin Buday vs Jake Collier

In [240]:
matchup_df = get_fighter_parameters("Martin Buday", "Jake Collier")
gamesSim(10000)

Martin Buday Decision Win  71.82 %
Martin Buday KO Win  27.73 %
Martin Buday Submission Win  0.0 %
Jake Collier Decision Win  0.45 %
Jake Collier KO Win  0.0 %
Jake Collier Submission Win  0.0 %
Tie  0.0 %


### Cody Durden vs Charles Johnson

In [241]:
matchup_df = get_fighter_parameters("Cody Durden", "Charles Johnson")
gamesSim(10000)

Cody Durden Decision Win  85.59 %
Cody Durden KO Win  1.16 %
Cody Durden Submission Win  0.0 %
Charles Johnson Decision Win  2.52 %
Charles Johnson KO Win  2.41 %
Charles Johnson Submission Win  8.32 %
Tie  0.0 %


### Stephanie Egger vs Irina Alekseeva - N/A (Irina Alekseeva first fight recorded on fightmetric)

### Journey Newson vs Marcus McGhee - N/A (Marcus McGhee first fight recorded on fightmetric)

### Hailey Cowan vs Jamey-Lyn Horth - N/A (Jamey-Lyn Horth first fight recorded on fightmetric)

## Predictions on UFC 288: Sterling vs. Cejudo

### Aljamain Sterling vs Henry Cejudo

In [244]:
matchup_df = get_fighter_parameters("Aljamain Sterling", "Henry Cejudo")
gamesSim(10000)

Aljamain Sterling Decision Win  27.69 %
Aljamain Sterling KO Win  0.29 %
Aljamain Sterling Submission Win  0.28 %
Henry Cejudo Decision Win  63.48 %
Henry Cejudo KO Win  8.26 %
Henry Cejudo Submission Win  0.0 %
Tie  0.0 %


### Belal Muhammad vs Gilbert Burns

In [245]:
matchup_df = get_fighter_parameters("Belal Muhammad", "Gilbert Burns")
gamesSim(10000)

Belal Muhammad Decision Win  93.46 %
Belal Muhammad KO Win  0.02 %
Belal Muhammad Submission Win  0.0 %
Gilbert Burns Decision Win  6.46 %
Gilbert Burns KO Win  0.04 %
Gilbert Burns Submission Win  0.02 %
Tie  0.0 %


### Jessica Andrade vs Yan Xiaonan

In [246]:
matchup_df = get_fighter_parameters("Jessica Andrade", "Yan Xiaonan")
gamesSim(10000)

Jessica Andrade Decision Win  1.57 %
Jessica Andrade KO Win  0.32 %
Jessica Andrade Submission Win  0.0 %
Yan Xiaonan Decision Win  98.11 %
Yan Xiaonan KO Win  0.0 %
Yan Xiaonan Submission Win  0.0 %
Tie  0.0 %


### Movsar Evloev vs Bryce Mitchell

In [247]:
matchup_df = get_fighter_parameters("Movsar Evloev", "Bryce Mitchell")
gamesSim(10000)

Movsar Evloev Decision Win  95.42 %
Movsar Evloev KO Win  0.0 %
Movsar Evloev Submission Win  0.0 %
Bryce Mitchell Decision Win  4.58 %
Bryce Mitchell KO Win  0.0 %
Bryce Mitchell Submission Win  0.0 %
Tie  0.0 %


### Kron Gracie vs Charles Jourdain

In [249]:
matchup_df = get_fighter_parameters("Kron Gracie", "Charles Jourdain")
gamesSim(10000)

Kron Gracie Decision Win  13.53 %
Kron Gracie KO Win  0.0 %
Kron Gracie Submission Win  42.38 %
Charles Jourdain Decision Win  42.72 %
Charles Jourdain KO Win  1.27 %
Charles Jourdain Submission Win  0.1 %
Tie  0.0 %


### Drew Dober vs Matt Frevola

In [ ]:
matchup_df = get_fighter_parameters("Kron Gracie", "Charles Jourdain")
gamesSim(10000)

### Kennedy Nzechukwu vs Devin Clark

In [250]:
matchup_df = get_fighter_parameters("Kennedy Nzechukwu", "Devin Clark")
gamesSim(10000)

Kennedy Nzechukwu Decision Win  0.8 %
Kennedy Nzechukwu KO Win  70.85 %
Kennedy Nzechukwu Submission Win  0.09 %
Devin Clark Decision Win  28.19 %
Devin Clark KO Win  0.07 %
Devin Clark Submission Win  0.0 %
Tie  0.0 %


### Khaos Williams vs Rolando Bedoya - N/A (Rolando Bedoya first fight recorded on fightmetric)

### Marina Rodriguez vs Virna Jandiroba

In [252]:
matchup_df = get_fighter_parameters("Marina Rodriguez", "Virna Jandiroba")
gamesSim(10000)

Marina Rodriguez Decision Win  97.9 %
Marina Rodriguez KO Win  0.24 %
Marina Rodriguez Submission Win  0.0 %
Virna Jandiroba Decision Win  0.41 %
Virna Jandiroba KO Win  0.65 %
Virna Jandiroba Submission Win  0.8 %
Tie  0.0 %


### Braxton Smith vs Parker Porter - N/A (Braxton Smith first fight recorded on fightmetric)

### Phil Hawes vs Ikram Aliskerov

In [255]:
matchup_df = get_fighter_parameters("Phil Hawes", "Ikram Aliskerov")
gamesSim(10000)

Phil Hawes Decision Win  0.0 %
Phil Hawes KO Win  0.01 %
Phil Hawes Submission Win  0.0 %
Ikram Aliskerov Decision Win  0.0 %
Ikram Aliskerov KO Win  0.02 %
Ikram Aliskerov Submission Win  99.97 %
Tie  0.0 %


### Rafael Estevam vs Zhalgas Zhumagulov

In [256]:
matchup_df = get_fighter_parameters("Rafael Estevam", "Zhalgas Zhumagulov")
gamesSim(10000)

Rafael Estevam Decision Win  2.14 %
Rafael Estevam KO Win  97.86 %
Rafael Estevam Submission Win  0.0 %
Zhalgas Zhumagulov Decision Win  0.0 %
Zhalgas Zhumagulov KO Win  0.0 %
Zhalgas Zhumagulov Submission Win  0.0 %
Tie  0.0 %


### Joseph Holmes vs Claudio Ribeiro

In [257]:
matchup_df = get_fighter_parameters("Joseph Holmes", "Claudio Ribeiro")
gamesSim(10000)

Joseph Holmes Decision Win  0.0 %
Joseph Holmes KO Win  31.85 %
Joseph Holmes Submission Win  27.19 %
Claudio Ribeiro Decision Win  4.97 %
Claudio Ribeiro KO Win  31.26 %
Claudio Ribeiro Submission Win  4.73 %
Tie  0.0 %


### Daniel Santos vs Johnny Munoz

In [258]:
matchup_df = get_fighter_parameters("Daniel Santos", "Johnny Munoz")
gamesSim(10000)

Daniel Santos Decision Win  2.45 %
Daniel Santos KO Win  47.41 %
Daniel Santos Submission Win  0.0 %
Johnny Munoz Decision Win  47.37 %
Johnny Munoz KO Win  0.0 %
Johnny Munoz Submission Win  2.77 %
Tie  0.0 %
